<h1>TAs please use your Personal Columbia Email in the API verfication</h1>
<p>I have allowed the following users to use the API</p>
<p>gl2664@columbia.edu	</p>
<p>jr4001@columbia.edu	</p>
<p>jromero0413@gmail.com	</p>
<p>jy3012@columbia.edu	</p>
<p>vad2134@columbia.edu	</p>
<p>xy2419@columbia.edu	</p>
<p>zk2172@columbia.edu	</p>
<p>zx2273@columbia.edu	</p>
<p>Please email jr4001@columbia.edu if you would like access and your email is not on this list </p>

In [1]:
import os
import time

In [2]:
os.getcwd()

'C:\\Users\\13473\\Desktop\\e4040-2020Fall-Project-WNZH-jr4001-xy2419-gl2664\\Ipynbs To Run'

In [3]:
wd_path = r'../'
os.chdir(wd_path)
wd_path=os.getcwd()

In [4]:
file_name = 'complete_universe_cleaned_normalized_jonn.csv'

In [5]:
import pandas as pd
import numpy as np

from datetime import datetime

In [6]:
df = pd.read_csv(file_name,dtype={'COMNAM':str,'NCUSIP':str,'PERMNO':str,'PERMCO':str,'SICCD':str,'TICKER_PERMNO':str},low_memory=False,index_col=0)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
df['DATE'] = pd.to_datetime(df['DATE'],format='%Y-%m-%d')

In [8]:
sector_map = {'0':'Agriculture, Forestry and Fishing','10':'Mining','11':'Mining','12':'Mining','13':'Mining','14':'Mining','15':'Construction','16':'Construction','17':'Construction','18':'not used','19':'not used','20':'Manufacturing','21':'Manufacturing','22':'Manufacturing','23':'Manufacturing','24':'Manufacturing','25':'Manufacturing','26':'Manufacturing','27':'Manufacturing','28':'Manufacturing','29':'Manufacturing','30':'Manufacturing','31':'Manufacturing','32':'Manufacturing','33':'Manufacturing','34':'Manufacturing','35':'Manufacturing','36':'Manufacturing','37':'Manufacturing','38':'Manufacturing','39':'Manufacturing','40':'Transportation, Communications, Electric, Gas and Sanitary service','41':'Transportation, Communications, Electric, Gas and Sanitary service','42':'Transportation, Communications, Electric, Gas and Sanitary service','44':'Transportation, Communications, Electric, Gas and Sanitary service','45':'Transportation, Communications, Electric, Gas and Sanitary service','46':'Transportation, Communications, Electric, Gas and Sanitary service','47':'Transportation, Communications, Electric, Gas and Sanitary service','48':'Transportation, Communications, Electric, Gas and Sanitary service','49':'Transportation, Communications, Electric, Gas and Sanitary service','50':'Wholesale Trade','51':'Wholesale Trade','52':'Retail Trade','53':'Retail Trade','54':'Retail Trade','55':'Retail Trade','56':'Retail Trade','57':'Retail Trade','58':'Retail Trade','59':'Retail Trade','60':'Finance, Insurance','61':'Finance, Insurance','62':'Finance, Insurance','63':'Finance, Insurance','64':'Finance, Insurance','65':'Real Estate','66':'Finance, Insurance','67':'Finance, Insurance','70':'Services','71':'Services','72':'Services','73':'Services','74':'Services','75':'Services','76':'Services','78':'Services','79':'Services','80':'Services','81':'Services','82':'Services','83':'Services','86':'Services','87':'Services','89':'Services','91':'Public Administration','92':'Public Administration','94':'Public Administration','95':'Public Administration','96':'Public Administration','97':'Public Administration','99':'Nonclassifiable'}

In [9]:
temp = pd.get_dummies(df['SICCD'].str[:2].replace(sector_map))
df = pd.concat([df, temp], axis=1, sort=False)

In [10]:
import gc

In [11]:
del temp 
gc.collect()
temp = None

In [12]:
df['m+1_returns'] = df['m+1']
df.loc[:,'m+1'] = df.groupby(['DATE'])['m+1'].transform(lambda x: x.rank(pct = True))

In [13]:
identifier_cols = ['DATE','COMNAM','NCUSIP','PERMNO','PERMCO','SICCD','TICKER_PERMNO']
cross_sectional_cols=['accru', 'adv_sale', 'aftret_eq', 'aftret_equity','aftret_invcapx', 'at_tu', 'bm', 'CAPEI', 'capital_rat', 'cash_debt',
       'cash_lt', 'cash_rat', 'cfm', 'curr_debt', 'curr_rat', 'de_rat','debt_assets', 'debt_at', 'debt_capit', 'debt_ebitd', 'debt_invcap',
       'divyield', 'dltt_be', 'equity_invcap', 'evm', 'gpm', 'GProf', 'intcov','intcov_rat', 'invt_act', 'lt_debt', 'lt_ppent', 'npm', 'ocf_lct',
       'opmad', 'opmbd', 'pay_tu', 'pcf', 'pe_ex', 'pe_inc', 'pretret_earnat','pretret', 'profit_lct', 'ps', 'ptb', 'ptpm', 'quick_rat', 'rd_sale',
       'rect_act', 'rect_tu', 'roa', 'roce', 'roe', 'sale_equity','sale_invcap', 'short_debt', 'staff_sale', 'totdebt_invcap']
sector_cols = ['Agriculture, Forestry and Fishing', 'Construction','Finance, Insurance', 'Manufacturing', 'Mining', 'Nonclassifiable',
       'Public Administration', 'Real Estate', 'Retail Trade', 'Services','Transportation, Communications, Electric, Gas and Sanitary service',
       'Wholesale Trade', 'not used']
time_series_cols= ['m-12','m-11', 'm-10', 'm-9', 'm-8', 'm-7', 'm-6', 'm-5', 'm-4', 'm-3', 'm-2','m-1']
all_cols = cross_sectional_cols + sector_cols + time_series_cols
Y = ['m+1']

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Concatenate, Input, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau

<h1>Regression Model</h1>

In [15]:
'''Hybrid Regression Model'''
time_series_inputs = Input(shape=(12,1))
cross_sectional_inputs = Input(shape=(83,))

h1_rets=LSTM(units=50, return_sequences=True)(time_series_inputs)
time_series_output=LSTM(units=30, return_sequences=False)(h1_rets) 

combined_features = Concatenate()([cross_sectional_inputs, time_series_output])

h1=Dense(128, activation='relu')(combined_features)
h2=Dense(64, activation='relu')(h1)
h3=Dense(32, activation='relu')(h2)
outputs=Dense(1, activation='sigmoid')(h3)

<h3>Here we are running our stocks through our models, these are out of sample</h3>

In [16]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [17]:
all_predictions = pd.DataFrame()
dates = df['DATE'].unique()
x = [df[all_cols],df[time_series_cols]]
y = df[Y]

for dt in dates[119:-1]:
    end_date = pd.to_datetime(dt)
    start_date = pd.to_datetime(pd.to_datetime(dt) - relativedelta(years=10))
    test_date = pd.to_datetime(pd.DataFrame({'year':[end_date.year],'month':[end_date.month],'day':[1]})).loc[0] + relativedelta(months=2) - relativedelta(days=1)
    
    print(start_date,end_date)
    print(test_date)
    
    model_file_name = 'hdf5_files\hybrid_reg_model_train_period_'+str(start_date.month)+'_'+str(start_date.year)+'_'+str(end_date.month)+'_'+str(end_date.year)+'.hdf5'
    
    hybrid_model=tf.keras.Model(inputs=[cross_sectional_inputs,time_series_inputs], outputs=outputs)

    opt=tf.keras.optimizers.Adam(
        learning_rate=0.005,
    )
    hybrid_model.compile(
        optimizer=opt, loss='binary_crossentropy',metrics=['binary_crossentropy','mean_squared_error','mean_absolute_error','mean_absolute_percentage_error']
    )
    hybrid_model.load_weights(model_file_name)
    
    mask_test=(df[['DATE']]==test_date)

    x_test, y_test= [i.loc[mask_test['DATE'],:] for i in x], y.loc[mask_test['DATE']]
    
    predictions = pd.DataFrame(hybrid_model.predict(x_test),index = y_test.index,columns=['Rank Predictions'])
    all_predictions = all_predictions.append(predictions)
    
    del predictions
    del x_test
    del y_test
    del mask_test
    del hybrid_model
    del model_file_name
    gc.collect()
    predictions = None
    x_test = None
    y_test = None
    mask_test = None
    hybrid_model = None
    model_file_name = None

1970-12-31 00:00:00 1980-12-31 00:00:00
1981-01-31 00:00:00


UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[functional_1/lstm/PartitionedCall]] [Op:__inference_predict_function_2111]

Function call stack:
predict_function -> predict_function -> predict_function


In [ ]:
all_predictions

In [ ]:
returns_df = df[['DATE','m+1_returns']].copy()
returns_df = returns_df.merge(all_predictions,left_index=True, right_index=True)

In [ ]:
from scipy.stats import zscore
returns_df['ZScore Predictions'] = returns_df.groupby(['DATE'])['Rank Predictions'].transform(lambda x : zscore(x,ddof=1))

In [ ]:
returns_dates =  returns_df['DATE'].unique()

In [ ]:
returns_df.head(10)

<h1>Model Factor Returns</h1>

<h3>For more information on factor investing</h3>
<a>https://en.wikipedia.org/wiki/Factor_investing</a>

In [ ]:
import statsmodels.api as sm

model_factor_returns = pd.DataFrame()

for dt in returns_dates:
    idx = (returns_df['DATE']==dt)
    
    X = returns_df.loc[idx,'ZScore Predictions']
    y = returns_df.loc[idx,'m+1_returns']
    
    model = sm.OLS(y, X)
    results = model.fit()
    
    model_factor_returns = model_factor_returns.append(pd.DataFrame(results.params[0]/100+1,index=[dt],columns=['Return']))

In [ ]:
model_factor_returns

In [ ]:
annual_returns = (model_factor_returns.resample('A').prod()-1)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Annual Returns', x=annual_returns.index, y=annual_returns['Return'])
])

fig.show()

<h1>Long Top 4% Short Bottom 4%</h1>

In [ ]:
percent = 0.04

model_ls_returns = pd.DataFrame()

for dt in returns_dates:
    idx = (returns_df['DATE']==dt)
    num = int(np.ceil(idx.sum() * percent))
    print(num)
    temp = returns_df.loc[idx].copy().sort_values(['Rank Predictions'])
    short = temp.head(num)
    long = temp.tail(num)
    
    ret = long['m+1_returns'].mean() - short['m+1_returns'].mean()
    model_ls_returns = model_ls_returns.append(pd.DataFrame(ret/100+1,index=[dt],columns=['Return']))

In [ ]:
annual_returns_2 = (model_ls_returns.resample('A').prod()-1)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Annual Returns', x=annual_returns_2.index, y=annual_returns_2['Return'])
])

fig.show()

<h1>Classification Model</h1>

In [ ]:
time_series_inputs = Input(shape=(12,1))
cross_sectional_inputs = Input(shape=(83,))

h1_rets=LSTM(units=50, return_sequences=True)(time_series_inputs)
time_series_output=LSTM(units=30, return_sequences=False)(h1_rets) 

combined_features = Concatenate()([cross_sectional_inputs, time_series_output])

h1=Dense(128, activation='relu')(combined_features)
h2=Dense(64, activation='relu')(h1)
h3=Dense(32, activation='relu')(h2)
outputs=Dense(5, activation='softmax')(h3)

In [ ]:
all_predictions = pd.DataFrame()
dates = df['DATE'].unique()

X = [df[all_cols],df[time_series_cols]]
y = pd.DataFrame(pd.get_dummies(pd.qcut(df[Y].values[:,0], 5, range(0,5))),index=df.index)

for dt in dates[119:-1]:
    end_date = pd.to_datetime(dt)
    start_date = pd.to_datetime(pd.to_datetime(dt) - relativedelta(years=10))
    test_date = pd.to_datetime(pd.DataFrame({'year':[end_date.year],'month':[end_date.month],'day':[1]})).loc[0] + relativedelta(months=2) - relativedelta(days=1)
    
    print(start_date,end_date)
    print(test_date)
    
    try:
        model_file_name = 'hdf5_files\hybrid_class_model_train_period_'+str(start_date.month)+'_'+str(start_date.year)+'_'+str(end_date.month)+'_'+str(end_date.year)+'.hdf5'
    
        hybrid_model=tf.keras.Model(inputs=[cross_sectional_inputs,time_series_inputs], outputs=outputs)

        opt=tf.keras.optimizers.Adam(
            learning_rate=0.005,
        )
        hybrid_model.compile(
            optimizer=opt, loss='CategoricalCrossentropy',metrics=['accuracy']
        )
        hybrid_model.load_weights(model_file_name)

        mask_test=(df[['DATE']]==test_date)

        x_test, y_test=[i.loc[mask_test['DATE'],:] for i in x], y.loc[mask_test['DATE']]

        predictions = pd.DataFrame(hybrid_model.predict(x_test),index = y_test.index,columns=[0,1,2,3,4])
        all_predictions = all_predictions.append(predictions)
    except:
        print('error')

In [ ]:
all_predictions

In [ ]:
returns_df_2 = df[['DATE','m+1_returns']].copy()
returns_df_2 = returns_df_2.merge(all_predictions,left_index=True, right_index=True)

In [ ]:
returns_dates =  returns_df_2['DATE'].unique()

In [ ]:
percent = 0.04

model_ls_returns_2 = pd.DataFrame()

for dt in returns_dates:
    idx = (returns_df_2['DATE']==dt)
    num = int(np.ceil(idx.sum() * percent))
    print(num)
    long = returns_df_2.loc[idx].copy().sort_values([4]).tail(num)
    short = returns_df_2.loc[idx].copy().sort_values([0]).tail(num)
    
    ret = long['m+1_returns'].mean() - short['m+1_returns'].mean()
    model_ls_returns_2 = model_ls_returns_2.append(pd.DataFrame(ret/100+1,index=[dt],columns=['Return']))

In [ ]:
annual_returns_3 = (model_ls_returns_2.resample('A').prod()-1)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Annual Returns', x=annual_returns_3.index, y=annual_returns_3['Return'])
])

fig.show()

<h1>Regression Model seems to have better Returns</h1>

In [ ]:
del df
df = None

In [ ]:
normalized_data = 'complete_universe_cleaned_normalized_jonn.csv'
auto_encoder_data = "Autoencoder_transformed_43_dim_data.csv"

In [ ]:
start_time = time.time()
df = pd.read_csv(normalized_data ,dtype={'COMNAM':str,'NCUSIP':str,'PERMNO':str,'PERMCO':str,'SICCD':str,'TICKER_PERMNO':str},low_memory=False,index_col=0)
df['DATE'] = pd.to_datetime(df['DATE'],format='%Y-%m-%d')
df = df.reset_index()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
auto = pd.read_csv(auto_encoder_data,low_memory=False,index_col=0)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
new_df = pd.concat([df[identifier_cols],auto,df[returns]], axis=1, sort=False)

In [ ]:
cross_sectional_cols = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42']
time_series_cols = ['m-12', 'm-11', 'm-10',
       'm-9', 'm-8', 'm-7', 'm-6', 'm-5', 'm-4', 'm-3', 'm-2', 'm-1']
Y = ['m+1']

In [ ]:
new_df.loc[:,'m+1'] = new_df.groupby(['DATE'])['m+1'].transform(lambda x: x.rank(pct = True))

In [ ]:
'''Model 4: LSTM + MLP Regression + Autoencoders'''
time_series_inputs = Input(shape=(12,1))
cross_sectional_inputs = Input(shape=(43,))

h1_rets=LSTM(units=50, return_sequences=True)(time_series_inputs)
time_series_output=LSTM(units=30, return_sequences=False)(h1_rets) 

combined_features = Concatenate()([cross_sectional_inputs, time_series_output])

h1=Dense(128, activation='relu')(combined_features)
h2=Dense(64, activation='relu')(h1)
h3=Dense(32, activation='relu')(h2)
outputs=Dense(1, activation='sigmoid')(h3)

hybrid_model2=tf.keras.Model(inputs=[cross_sectional_inputs,time_series_inputs], outputs=outputs)

opt=tf.keras.optimizers.Adam(
    learning_rate=0.005,
)
hybrid_model2.compile(
    optimizer=opt, loss='binary_crossentropy',metrics=['binary_crossentropy','mean_squared_error','mean_absolute_error','mean_absolute_percentage_error']
)

In [ ]:
all_predictions = pd.DataFrame()
dates = df['DATE'].unique()
x = [df[cross_sectional_cols],df[time_series_cols]]
y = df[Y]

for dt in dates[119:-1]:
    end_date = pd.to_datetime(dt)
    start_date = pd.to_datetime(pd.to_datetime(dt) - relativedelta(years=10))
    test_date = pd.to_datetime(pd.DataFrame({'year':[end_date.year],'month':[end_date.month],'day':[1]})).loc[0] + relativedelta(months=2) - relativedelta(days=1)
    
    print(start_date,end_date)
    print(test_date)
    
    model_file_name = 'hdf5_files\hybrid_reg_model_w_auto_train_period_'+str(start_date.month)+'_'+str(start_date.year)+'_'+str(end_date.month)+'_'+str(end_date.year)+'.hdf5'
    
    hybrid_model2=tf.keras.Model(inputs=[cross_sectional_inputs,time_series_inputs], outputs=outputs)

    opt=tf.keras.optimizers.Adam(
        learning_rate=0.005,
    )
    hybrid_model2.compile(
        optimizer=opt, loss='binary_crossentropy'
    )
    hybrid_model2.load_weights(model_file_name)
    
    mask_test=(df[['DATE']]==test_date)

    x_test, y_test= [i.loc[mask_test['DATE'],:] for i in x], y.loc[mask_test['DATE']]
    
    predictions = pd.DataFrame(hybrid_model.predict(x_test),index = y_test.index,columns=['Rank Predictions'])
    all_predictions = all_predictions.append(predictions)
    
    del predictions
    del x_test
    del y_test
    del mask_test
    del hybrid_model2
    del model_file_name
    gc.collect()
    predictions = None
    x_test = None
    y_test = None
    mask_test = None
    hybrid_model2 = None
    model_file_name = None

In [ ]:
all_predictions